### Relative Affordability Calculaton

In [145]:
import pandas as pd
import numpy as np 
import geopy.distance
from geopy.distance import geodesic
import warnings

pd.set_option('display.max_columns', None)

warnings.filterwarnings("ignore")

### Import Data

In [181]:
zips_data = pd.read_csv('../nicole/clean_data/final_zip_data.csv')
zips_data.head(5)

,ZipCode,State,County,CBSA_Name,CountyFIPS,StateFIPS,Population,HouseholdsPerZipCode,WhitePopulation,BlackPopulation,HispanicPopulation,AsianPopulation,IndianPopulation,OtherPopulation,MalePopulation,FemalePopulation,AverageHouseValue,IncomePerHousehold,MedianAge,NumberOfBusinesses,over_65_ratio,school_rating,Hosp_count,Zip_count,HPI,Demand_score,Supply_score,listviews_vs_US,med_days_on_mkt,nielson_rank,UE_rate,Zillow_HVF,BEA_percap_income,Survey_Date,FIPS_State,FIPS_County,Region_Code,Division_Code,State_y,state_abbrev,County_Name,one_unit_buildings_rep,one_unit_units_rep,one_unit_value_reps,two_unit_buildings_rep,two_unit_units_rep,two_unit_value_rep,three_unit_building_rep,three_unit_untis_rep,three_unit_value_rep,five_unit_building_rep,five_unit_unit_rep,five_unit_value_rep
0,20001,DC,DISTRICT OF COLUMBIA,Washington Arlington Alexandria,1,11,38551,16500,13671,20682,3555,3147,488,1952,18912,19639,692900,116195,30.3,1978,0.078726,5.727273,NaN,783.0,330.6,5.351171,70.234114,0.4816,41.0,7.0,4.4,7.1,86567.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20002,DC,DISTRICT OF COLUMBIA,Washington Arlington Alexandria,1,11,52370,22973,17522,33303,2299,1312,596,999,24896,27474,637300,90424,34.9,1674,0.085151,4.851852,NaN,783.0,330.6,5.351171,70.234114,0.4816,41.0,7.0,4.4,8.2,86567.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20003,DC,DISTRICT OF COLUMBIA,Washington Arlington Alexandria,1,11,26454,11724,15295,10111,1355,1043,296,460,14054,12400,806500,134438,34.1,1227,0.101423,5.416667,NaN,783.0,330.6,5.351171,70.234114,0.4816,41.0,7.0,4.4,8.3,86567.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20004,DC,DISTRICT OF COLUMBIA,Washington Arlington Alexandria,1,11,1622,1127,1321,131,109,183,17,27,834,788,594600,146822,37.9,961,0.117005,8.000000,NaN,783.0,330.6,5.351171,70.234114,0.4816,41.0,7.0,4.4,5.8,86567.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20005,DC,DISTRICT OF COLUMBIA,Washington Arlington Alexandria,1,11,12775,8316,8478,2157,2142,1232,128,1228,7000,5775,536700,93956,32.5,2366,0.090953,6.000000,NaN,783.0,330.6,5.351171,70.234114,0.4816,41.0,7.0,4.4,6.5,86567.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [188]:
listings_data = pd.read_csv('../nicole/clean_data/redfin_cleaned_stage_1.csv')
listings_data.head(3)
#listings_data['STATE OR PROVINCE'].unique()

,Unnamed: 0,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING),SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,zip
0,10730,MLS Listing,NaN,Condo/Co-op,810 O St NW #201,Washington,DC,20001.0,899900.0,2.0,2.0,OLD CITY II,1282.0,NaN,2019.0,250.0,702.0,972.0,Active,NaN,NaN,https://www.redfin.com/DC/Washington/810-O-St-...,BRIGHT MLS,DCDC514770,N,Y,38.908332,-77.023611,20001
1,147,MLS Listing,NaN,Townhouse,29 Q St NW,Washington,DC,20001,1175000.0,4.0,3.5,OLD CITY II,2313.0,1122.0,1900.0,8.0,508.0,NaN,Active,NaN,NaN,https://www.redfin.com/DC/Washington/29-Q-St-N...,BRIGHT MLS,DCDC514598,N,Y,38.911332,-77.011364,20001
2,574,MLS Listing,NaN,Condo/Co-op,815 T St NW Unit A,Washington,DC,20001,825000.0,3.0,2.5,OLD CITY II,1577.0,NaN,1900.0,68.0,523.0,198.0,Active,NaN,NaN,https://www.redfin.com/DC/Washington/815-T-St-...,BRIGHT MLS,DCDC2016186,N,Y,38.915751,-77.023640,20001


In [ ]:
#built function that returns:
lat/long, closest centroid, distance to that centroid, sale_price, year built

In [186]:
list((listings_data['CITY'].unique()))

['Washington',
 nan,
 'Bostic',
 'Aldie',
 'Amissville',
 'Newberry',
 'Manassas',
 'Manassas Park',
 'Marshall',
 'Middleburg',
 'Catlett',
 'Centreville',
 'Clifton',
 'Paeonian Springs',
 'Paris',
 'Purcellville',
 'Hillsboro',
 'Bluemont',
 'Bristow',
 'Broad Run',
 'Powhatan',
 'Round Hill',
 'Catharpin',
 'Delaplane',
 'Ashburn',
 'Broadlands',
 'Brambleton',
 'Chantilly',
 'South Riding',
 'Gainesville',
 'Hamilton',
 'Lincoln',
 'Sterling',
 'Dulles',
 'Haymarket',
 'Herndon',
 'Oak Hill',
 'Leesburg',
 'Lansdowne',
 'Lovettsville',
 'Nokesville',
 'Upperville',
 'Warrenton',
 'Reston',
 'Waterford',
 'The Plains',
 'Waldorf',
 'Abell',
 'Accokeek',
 'Aquasco',
 'Avenue',
 'Bel Alton',
 'Brandywine',
 'Broomes Island',
 'Bryans Road',
 'Bryantown',
 'Bushwood',
 'California',
 'Callaway',
 'Chaptico',
 'Charlotte Hall',
 'Cheltenham',
 'Clements',
 'Cobb Island',
 'Coltons Point',
 'Dameron',
 'Drayden',
 'Faulkner',
 'Great Mills',
 'Hollywood',
 'Hughesville',
 'Huntingtown',

In [189]:
smh_data = pd.read_csv('../nicole/clean_data/SMH_Projects.csv')
smh_data['MedianSalesPrice'] = (smh_data['MinBaseSalesPrice'] + smh_data['MaxBaseSalesPrice'])/2
smh_data.head(15)

#smh_data['MetroAreaID'].unique()

,ProjectGroupID,AltProjectID,ProjectID,ElementName,Division,MetroAreaID,Latitude,Longitude,Address,city,State,state_fips,ZipCode,County,MetroAreaTitle,CountyName,county_fips,cbsa_code,cbsa_name,msa_name,ProductID,ProductDescription,Active,MinSquareFootage,MaxSquareFootage,MinBaseSalesPrice,MaxBaseSalesPrice,ProductTypeID,NumberBedrooms,NumberFullBaths,NumberHalfBaths,NumberGarageSpaces,MedianSalesPrice
0,MST,721,Main Street Townes at Lilburn TH,Main Street Townes at Lilburn TH (721),Atlanta,1007,33.891702,-84.140783,346 Main St NW,Lilburn,GA,13.0,30047.0,29.0,Atlanta,Gwinnett,135.0,12060.0,Atlanta-Sandy Springs-Alpharetta GA,Atlanta GA MSA,001562,Dupree,Y,2113,2191,385900.0,393500.0,NaN,NaN,NaN,NaN,NaN,389700.0
1,LD,LD,West Broad Landing 2o2,West Broad Landing 2o2 (LD),Richmond,1006,37.616200,-77.521000,"2636 Lassen Walk, Unit A",Henrico,VA,51.0,23294.0,20.0,Richmond,Henrico,87.0,40060.0,Richmond VA,Richmond-Petersburg VA MSA,279,Paxton,Y,2452,2855,354990.0,355790.0,1.0,3.0,2.0,1.0,1.0,355390.0
2,SCK,SCK3,Sage Creek SFD,Sage Creek SFD (SCK3),Aiken/Augusta,1013,33.593076,-81.824541,1182 Sapphire Drive,Graniteville,SC,45.0,29829.0,37.0,Aiken/North Augusta,Aiken,3.0,12260.0,Augusta-Richmond County GA-SC,Augusta-Aiken GA-SC MSA,001420,Barlow,Y,2273,2306,236900.0,244900.0,4.0,3.0,2.0,1.0,2.0,240900.0
3,VST,VST3,Vancouver Station at Trolley Run SFD,Vancouver Station at Trolley Run SFD (VST3),Aiken/Augusta,1013,33.582816,-81.776370,507 Burnaby Court,Aiken,SC,45.0,29801.0,37.0,Aiken/North Augusta,Aiken,3.0,12260.0,Augusta-Richmond County GA-SC,Augusta-Aiken GA-SC MSA,001423,Santee,Y,1295,1593,210900.0,213900.0,4.0,3.0,2.0,0.0,2.0,212400.0
4,VST,VST3,Vancouver Station at Trolley Run SFD,Vancouver Station at Trolley Run SFD (VST3),Aiken/Augusta,1013,33.582816,-81.776370,507 Burnaby Court,Aiken,SC,45.0,29801.0,37.0,Aiken/North Augusta,Aiken,3.0,12260.0,Augusta-Richmond County GA-SC,Augusta-Aiken GA-SC MSA,001421,Summerton,Y,2386,2386,244900.0,250900.0,4.0,3.0,2.0,1.0,0.0,247900.0
5,SUM,663,Sumner Village SFD,Sumner Village SFD (663),Charleston,1008,32.898400,-79.997900,1414 Sumner Avenue,North Charleston,SC,45.0,29406.0,31.0,Charleston,Charleston,19.0,16700.0,Charleston-North Charleston SC,Charleston-North Charleston SC MSA,u14,Buist SLAB,Y,1678,1700,284900.0,294900.0,NaN,NaN,NaN,NaN,NaN,289900.0
6,WDB,WDB,Woodborne Preserve,Woodborne Preserve (WDB),Northern Virginia,1000,38.797253,-77.652348,James Madison Highway and Thoroughfare Road,Gainesville,VA,51.0,20155.0,9.0,Northern Virginia,Prince William,153.0,47900.0,Washington-Arlington-Alexandria DC-VA-MD-WV,Washington-Baltimore DC-MD-VA-WV CMSA,A35,Kasey BSMT,Y,3342,6648,877490.0,892490.0,4.0,4.0,3.0,1.0,2.0,884990.0
7,CRD,CSP3,Clairbourne Springs SFD,Clairbourne Springs SFD (CSP3),Aiken/Augusta,1013,33.523745,-81.876585,6078 Whirlaway Road,Graniteville,SC,45.0,29829.0,37.0,Aiken/North Augusta,Aiken,3.0,12260.0,Augusta-Richmond County GA-SC,Augusta-Aiken GA-SC MSA,001421,Summerton,Y,2386,2386,243900.0,249900.0,4.0,3.0,2.0,1.0,2.0,246900.0
8,SOD,SOD3,Summer Orchard SFD,Summer Orchard SFD (SOD3),Columbia,1010,33.917912,-81.328175,Summer Orchard Drive,Lexington,SC,45.0,29072.0,52.0,Columbia,Lexington,63.0,17900.0,Columbia SC,Columbia SC MSA,001421,Summerton,Y,2386,2386,239900.0,243900.0,NaN,NaN,NaN,NaN,NaN,241900.0
9,EY,EYM3,Embrey Mill 2 SFD,Embrey Mill 2 SFD (EYM3),Northern Virginia,1000,38.430500,-77.428900,510 Sourwood Court,Stafford,VA,51.0,22554.0,11.0,Northern Virginia,Stafford,179.0,47900.0,Washington-Arlington-Alexandria DC-VA-MD-WV,Washington-Baltimore DC-MD-VA-WV CMSA,785,Middleton,Y,3301,5757,637990.0,643990.0,4.0,4.0,2.0,1.0,2.0,640990.0


In [199]:
smh_data.groupby('MetroAreaID').agg({'ZipCode':pd.Series.nunique}).reset_index()

#agg({'placeID':pd.Series.nunique}).reset_index()

,MetroAreaID,ZipCode
0,1000,10
1,1001,9
2,1003,5
3,1004,5
4,1006,6
5,1007,8
6,1008,6
7,1009,1
8,1010,7
9,1011,4


In [158]:
centroids = pd.read_csv('../project/centroid_dummy.csv')
#centroids.head(2)

#### Creating some merged tables

In [159]:
#lat/long and Median Sales Price for each SMH property
smh_geo = smh_data[['MetroAreaID', 'Latitude', 'Longitude', 'MedianSalesPrice']]
smh_geo.head()

,MetroAreaID,Latitude,Longitude,MedianSalesPrice
0,1007,33.891702,-84.140783,389700.0
1,1006,37.616200,-77.521000,355390.0
2,1013,33.593076,-81.824541,240900.0
3,1013,33.582816,-81.776370,212400.0
4,1013,33.582816,-81.776370,247900.0


In [162]:
#merge centroid table with smh_geo table & drop NAs
smh_centroid = smh_geo.merge(centroids, on='MetroAreaID', how="left")
smh_centroid.columns = ["MetroAreaID", "Latitude" ,"Longitude",'MedianSalesPrice', "centroid_lat", "centroid_long"]

smh_centroid.dropna(inplace = True)
smh_centroid

,MetroAreaID,Latitude,Longitude,MedianSalesPrice,centroid_lat,centroid_long
0,1007,33.891702,-84.140783,389700.0,33.748783,-84.388168
1,1006,37.616200,-77.521000,355390.0,37.541290,-77.434769
2,1013,33.593076,-81.824541,240900.0,33.507600,-81.860400
3,1013,33.582816,-81.776370,212400.0,33.507600,-81.860400
4,1013,33.582816,-81.776370,247900.0,33.507600,-81.860400
...,...,...,...,...,...,...
445,1001,38.300400,-76.549900,327990.0,39.045753,-76.641273
446,1004,35.794500,-78.940100,483990.0,35.884766,-78.625053
448,1000,38.907833,-77.496722,866490.0,38.805100,-77.047000
449,1000,38.797253,-77.652348,940490.0,38.805100,-77.047000


In [160]:
#dict of centroid lat/long for each SMH community/metro area ID
#smh_geo_dict = smh_geo.set_index('MetroAreaID').T.to_dict('list')
#centroids_dict = centroids.set_index('MetroAreaID').T.to_dict('list')
#centroids
#centroids_dict

In [201]:
smh_centroid['lat_long'] = list(zip(smh_centroid.Latitude,smh_centroid.Longitude))
smh_centroid['centroid'] = list(zip(smh_centroid.centroid_lat,smh_centroid.centroid_long))
smh_centroid = smh_centroid.drop(["Latitude", "Longitude", "centroid_lat", "centroid_long"], axis=1)
smh_centroid['distance'] = smh_centroid.apply(lambda x: geopy.distance.geodesic(x.lat_long, x.centroid), axis = 1)

#df['col_3'] = df.apply(lambda x: f(x.col_1, x.col_2), axis=1)

#### Distance to Centroid & Median Sale Prices

In [202]:
#list of every SMH property and distance to centroid
smh_centroid.head(30)

,MetroAreaID,MedianSalesPrice,lat_long,centroid,distance
0,1007,389700.0,"(33.891702, -84.140783)","(33.748783, -84.388168)",27.85372953251022 km
1,1006,355390.0,"(37.6162, -77.521)","(37.54129, -77.434769)",11.275803895831249 km
2,1013,240900.0,"(33.593076, -81.824541)","(33.5076, -81.8604)",10.04839088810054 km
3,1013,212400.0,"(33.582816, -81.77637)","(33.5076, -81.8604)",11.423813153281548 km
4,1013,247900.0,"(33.582816, -81.77637)","(33.5076, -81.8604)",11.423813153281548 km
5,1008,289900.0,"(32.8984, -79.9979)","(32.776566, -79.930923)",14.895773216251829 km
6,1000,884990.0,"(38.7972531, -77.6523484)","(38.8051, -77.047)",52.59271546370725 km
7,1013,246900.0,"(33.5237447, -81.8765853)","(33.5076, -81.8604)",2.3382963395454497 km
8,1010,241900.0,"(33.917912, -81.328175)","(34.0007, -81.0348)",28.62901156133804 km
9,1000,640990.0,"(38.4305, -77.4289)","(38.8051, -77.047)",53.2485419990901 km


## Inputs

#### Average Resale Price:
- Average resale price of the competitor houses within the radius of selected distance  from the SMH community. (Previous Year Closing price) 

In [166]:
price_dict = smh_data.groupby('MetroAreaID')['MedianSalesPrice'].mean().round().astype(int).to_dict()
smh_data['MetroIDAveragePrice'] = smh_data['MetroAreaID'].map(price_dict)

smh_data

,ProjectGroupID,AltProjectID,ProjectID,ElementName,Division,MetroAreaID,Latitude,Longitude,Address,city,State,state_fips,ZipCode,County,MetroAreaTitle,CountyName,county_fips,cbsa_code,cbsa_name,msa_name,ProductID,ProductDescription,Active,MinSquareFootage,MaxSquareFootage,MinBaseSalesPrice,MaxBaseSalesPrice,ProductTypeID,NumberBedrooms,NumberFullBaths,NumberHalfBaths,NumberGarageSpaces,MedianSalesPrice,MetroIDAveragePrice
0,MST,721,Main Street Townes at Lilburn TH,Main Street Townes at Lilburn TH (721),Atlanta,1007,33.891702,-84.140783,346 Main St NW,Lilburn,GA,13.0,30047.0,29.0,Atlanta,Gwinnett,135.0,12060.0,Atlanta-Sandy Springs-Alpharetta GA,Atlanta GA MSA,001562,Dupree,Y,2113,2191,385900.0,393500.0,NaN,NaN,NaN,NaN,NaN,389700.0,442563
1,LD,LD,West Broad Landing 2o2,West Broad Landing 2o2 (LD),Richmond,1006,37.616200,-77.521000,"2636 Lassen Walk, Unit A",Henrico,VA,51.0,23294.0,20.0,Richmond,Henrico,87.0,40060.0,Richmond VA,Richmond-Petersburg VA MSA,279,Paxton,Y,2452,2855,354990.0,355790.0,1.0,3.0,2.0,1.0,1.0,355390.0,342769
2,SCK,SCK3,Sage Creek SFD,Sage Creek SFD (SCK3),Aiken/Augusta,1013,33.593076,-81.824541,1182 Sapphire Drive,Graniteville,SC,45.0,29829.0,37.0,Aiken/North Augusta,Aiken,3.0,12260.0,Augusta-Richmond County GA-SC,Augusta-Aiken GA-SC MSA,001420,Barlow,Y,2273,2306,236900.0,244900.0,4.0,3.0,2.0,1.0,2.0,240900.0,284217
3,VST,VST3,Vancouver Station at Trolley Run SFD,Vancouver Station at Trolley Run SFD (VST3),Aiken/Augusta,1013,33.582816,-81.776370,507 Burnaby Court,Aiken,SC,45.0,29801.0,37.0,Aiken/North Augusta,Aiken,3.0,12260.0,Augusta-Richmond County GA-SC,Augusta-Aiken GA-SC MSA,001423,Santee,Y,1295,1593,210900.0,213900.0,4.0,3.0,2.0,0.0,2.0,212400.0,284217
4,VST,VST3,Vancouver Station at Trolley Run SFD,Vancouver Station at Trolley Run SFD (VST3),Aiken/Augusta,1013,33.582816,-81.776370,507 Burnaby Court,Aiken,SC,45.0,29801.0,37.0,Aiken/North Augusta,Aiken,3.0,12260.0,Augusta-Richmond County GA-SC,Augusta-Aiken GA-SC MSA,001421,Summerton,Y,2386,2386,244900.0,250900.0,4.0,3.0,2.0,1.0,0.0,247900.0,284217
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
411,DH,DH,Woodhall,Woodhall (DH),Raleigh,1004,35.794500,-78.940100,165 Ferrell Rd W,Apex,NC,37.0,27523.0,24.0,Raleigh/Durham,Chatham,37.0,20500.0,Durham-Chapel Hill NC,Raleigh-Durham-Chapel Hill NC MSA,A06,Buchanan II CRWL,Y,1689,2797,478990.0,488990.0,4.0,3.0,2.0,0.0,2.0,483990.0,507030
412,HST,HST3,Holliston SFD,Holliston SFD (HST3),Greenville/Spartanburg,1012,34.725073,-82.222818,6 Ashborne Lane,Simpsonville,SC,45.0,29681.0,41.0,Greenville/Spartanburg,Greenville,45.0,24860.0,Greenville-Anderson SC,Greenville-Spartanburg-Anderson SC MSA,001414,Tilbury,Y,1296,1296,241900.0,243900.0,4.0,3.0,2.0,0.0,2.0,242900.0,331341
413,PLH,PLH,Poland Hill SFD,Poland Hill SFD (PLH),Northern Virginia,1000,38.907833,-77.496722,25588 Poland Rd,Chantilly,VA,51.0,20152.0,3.0,Northern Virginia,Loudoun,107.0,47900.0,Washington-Arlington-Alexandria DC-VA-MD-WV,Washington-Baltimore DC-MD-VA-WV CMSA,001372,Wayne,Y,2907,4375,860990.0,871990.0,4.0,3.0,2.0,1.0,2.0,866490.0,700512
414,WDB,WDB,Woodborne Preserve,Woodborne Preserve (WDB),Northern Virginia,1000,38.797253,-77.652348,James Madison Highway and Thoroughfare Road,Gainesville,VA,51.0,20155.0,9.0,Northern Virginia,Prince William,153.0,47900.0,Washington-Arlington-Alexandria DC-VA-MD-WV,Washington-Baltimore DC-MD-VA-WV CMSA,A90,Russell,Y,3860,6787,927490.0,953490.0,4.0,4.0,3.0,1.0,2.0,940490.0,700512


#### Average Stanley Martin Base Price: 
- Average base price of all SMH homes in the selected SMH Community. 

In [39]:
#list of average sale price of each SMH Community 
smh = smh_data.groupby(['MetroAreaID']).agg({'MedianSalesPrice': ['mean']}).reset_index()
smh.columns = ['MetroAreaID', 'Average Sale Price']
smh.head()

,MetroAreaID,Average Sale Price
0,1000,700512.282609
1,1001,541461.428571
2,1003,492651.590909
3,1004,507030.000000
4,1006,342768.846154


#### Average Resale Year Built: 
- Average year when the resale homes were built. (Example 1992) 

In [ ]:
Sell Year = year they want to sell 2023
community sell in --> metro area code 
distance --> radius length 

10 properties --> 1997
2023 - 1997 = 26

then lookup table to find 

## Calculations

##### Age of Resales: 
- Average Resale Year Built — Current Year (e.g., 2021 -1992 = 29) 

##### Expected Resale Premium: 
- Based on the age of Resales, this is derived from lookup table #1. Example: For  age band of 21-30, Resale Premium is 40% 

##### Actual Resale Premium: 
- (Average Stanley Martin Base Price — Average Resale Price)/ Average Resale Price 

##### Score: 
- No score if there is no Actual Resale Premium. 
- If Actual Resale Premium > Expected Resale  Premium, then 1. 
- Else, if Actual Resale Premium< Expected Resale Premium then 3, else 2. 

##### Results: 
- Is Pricing Above or Below the Expected Premium to Resales? 
- To get this Premium to Resales value is looked up to Resale Premium Analysis table. (Table 2) Relative Affordability Ranking: 
- Based on the Score from the Calculations above. Lookup Table #3 and obtain the ranking.


In [ ]:
ideas:
map, show radius , show score 
inputs: radius length, input address (come with community), 